In [10]:
import math
import torch
from torch.nn.parameter import Parameter # 也可直接调用nn.Parameter
from torch.nn.modules.module import Module # 也可直接调用nn.Module

# 注意：torch.FloatTensor生成的元素数值非常接近0;torch.Long生成的元素数值非常大
class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        # self.weight2 = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter("bias", None) #
        self.reset_parameters() # 此处表示生成变量后(即上面的语句运行后),将会进行变量初始化(即执行该语句)

    def reset_parameters(self): # 经测试,重写可覆盖
        stdv = 1/math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        # self.weight2.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # support = torch.mm(input, self.weight)    # 尝试：此行与下一行交换顺序
        # output = torch.spmm(adj, support) # torch.spmm支持sparse在前,dense在后的矩阵乘法
        support = torch.spmm(adj, input)
        output = torch.mm(support, self.weight)
        if self.bias is not None:         # adj是稀疏矩阵
            return output +self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__+"("+str(self.in_features)+"->"+str(self.out_features)+")"

# class Readout(Module):
#     def __init__(self, in_features):
#         super(Readout, self).__init__()
#         self.in_features = in_features
#         self.weight = Parameter(torch.FloatTensor(in_features, in_features))
#         self.reset_parameters()
#
#     def reset_parameters(self):
#         stdv = 1/math.sqrt(self.weight.size(1))
#         self.weight.data.uniform_(-stdv, stdv)
#
#     def forward(self, input):
#         input = torch.sum(input, 0) # 此处不确定
#         return torch.mm(input, self.weight)

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        # self.gc2 = GraphConvolution(nhid, nhid)
        self.dropout = dropout
        # self.rd = Readout(nhid)
        # self.fc1 = nn.Linear(nhid, 1)
        # nn.init.kaiming_normal_(self.fc1.weight)

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)
        # x = F.relu(self.gc2(x, adj))
        # x = F.dropout(x, self.dropout, training=self.training)
        # x = F.relu(self.rd(x))
        # x = self.fc(x)
        # return x

In [12]:
# 理解one-hot编码的过程：
import numpy as np
a = [3, 9, 6]
b = set(a)
print(b)
c = sorted(list(set(a)))
print(c)
d = {t: np.identity(len(b))[i, :] for i, t in enumerate(b)}
# d = {t: np.identity(len(b))[i, :] for i, t in enumerate(c)}
print(d)
e = np.array(list(map(d.get, a)), dtype=np.int32) # c.get为字典内置函数
print(e)
print(map(d.get, a))
print(list(map(d.get, a)))

{9, 3, 6}
[3, 6, 9]
{9: array([1., 0., 0.]), 3: array([0., 1., 0.]), 6: array([0., 0., 1.])}
[[0 1 0]
 [1 0 0]
 [0 0 1]]
[array([0., 1., 0.]), array([1., 0., 0.]), array([0., 0., 1.])]


In [13]:
# 理解np.genfromtxt和sp.csr_matrix的处理：
import numpy as np
import scipy.sparse as sp
idx_features_labels = np.genfromtxt("{}{}.content".format("./cora/", "cora"), dtype=np.dtype(str))
print(idx_features_labels)
mid_idx_features_labels = idx_features_labels[:, 1:-1]
after_idx_features_labels = np.zeros(mid_idx_features_labels.shape)
for i in range(mid_idx_features_labels.shape[0]):
    for j in range(mid_idx_features_labels.shape[1]):
        after_idx_features_labels[i][j] = int(mid_idx_features_labels[i][j])
features = sp.csr_matrix(after_idx_features_labels, dtype=np.float32)
print(after_idx_features_labels)
# print(features)
print(features.shape)

[['31336' '0' '0' ... '0' '0' 'Neural_Networks']
 ['1061127' '0' '0' ... '0' '0' 'Rule_Learning']
 ['1106406' '0' '0' ... '0' '0' 'Reinforcement_Learning']
 ...
 ['1128978' '0' '0' ... '0' '0' 'Genetic_Algorithms']
 ['117328' '0' '0' ... '0' '0' 'Case_Based']
 ['24043' '0' '0' ... '0' '0' 'Neural_Networks']]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2708, 1433)


In [14]:
# 注意此处a中的元素为字符串，且直接在a上修改覆盖无法生效，故要创建另一个与a的shape一致的变量
a = idx_features_labels[1:5, 1:5]
b = np.zeros(a.shape)
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        # a[i][j] = int(a[i][j])
        b[i][j] = int(a[i][j])

# print(a[0][0]-1)
print(b[0][0]-1)

-1.0


In [15]:
import numpy as np
import scipy.sparse as sp
import torch

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c:np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def load_data(path="./cora/", dataset="cora"):
    print("Loading {} dataset...".format(dataset))
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j:i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T>adj) - adj.multiply(adj.T>adj) # 最后减的那一项目的是去除负边

    features = normalize(features)
    # adj = normalize(adj + sp.eye(adj.shape[0]))
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    return adj, features, labels, idx_train, idx_val, idx_test

def normalize(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # 此处得到一个对角矩阵
    mx = r_mat_inv.dot(mx) # 注意.dot为矩阵乘法,不是对应元素相乘
    return mx

def normalize_adj(mx):
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    mid = np.dot(r_mat_inv_sqrt, mx)
    return np.dot(mid, r_mat_inv_sqrt)

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels).reshape(labels.shape)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = sparse_mx.shape
    return torch.sparse.FloatTensor(indices, values, shape)

In [27]:
# 理解.todense()
a = np.array([[0,3,2],[0,4,5],[1,3,6],[1,5,7],[2,4,10]])
b = sp.coo_matrix(arg1=(a[:, 2], (a[:, 0], a[:, 1])), shape=(7,7), dtype=np.float32)
c = b.todense()
print(a)
print("-"*50)
print(b)
print("-"*50)
print(c)

[[ 0  3  2]
 [ 0  4  5]
 [ 1  3  6]
 [ 1  5  7]
 [ 2  4 10]]
--------------------------------------------------
  (0, 3)	2.0
  (0, 4)	5.0
  (1, 3)	6.0
  (1, 5)	7.0
  (2, 4)	10.0
--------------------------------------------------
[[ 0.  0.  0.  2.  5.  0.  0.]
 [ 0.  0.  0.  6.  0.  7.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [28]:
# 理解np.where
labels = encode_onehot(idx_features_labels[:, -1])
print(labels)
print(np.where(labels)[0])
print(np.where(labels)[1])
a = [[1, 4, 0, -2, 0, 3, 0, 0, -5],[1, 4, 0, 2, 0, 3, 0, 0, 5]]
print(np.where(a)[0])
print(np.where(a)[1])
print(torch.tensor(a).double())

[[0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
[   0    1    2 ... 2705 2706 2707]
[2 0 6 ... 4 3 2]
[0 0 0 0 0 1 1 1 1 1]
[0 1 3 5 8 0 1 3 5 8]
tensor([[ 1.,  4.,  0., -2.,  0.,  3.,  0.,  0., -5.],
        [ 1.,  4.,  0.,  2.,  0.,  3.,  0.,  0.,  5.]], dtype=torch.float64)


In [29]:
# 理解.max(1)[1], .type_as()
b = torch.rand(3,5)
print(b)
print(b.max(1))
print(b.max(1)[1].type_as(torch.tensor(labels)))
print(labels.dtype)

tensor([[0.4187, 0.6349, 0.2859, 0.0057, 0.0730],
        [0.7526, 0.4721, 0.8668, 0.9557, 0.6298],
        [0.1914, 0.0729, 0.1411, 0.7865, 0.2367]])
torch.return_types.max(
values=tensor([0.6349, 0.9557, 0.7865]),
indices=tensor([1, 3, 3]))
tensor([1, 3, 3], dtype=torch.int32)
int32


In [31]:
# 理解.astype(np.float32)
c = np.array(b).astype(np.float32) # 只有np.array才能用astype
print(c.dtype)
print(b.size())

float32
torch.Size([3, 5])


In [33]:
# 理解函数normalize的细节
a = np.array([[1, 0, 5], [0, 2, 0], [0, 0, 3]], dtype=np.float64)
print(a)
b = np.array(a.sum(1))
print(b)
b = np.power(b, -1).flatten()
b[np.isinf(b)] = 0.
print(b)
c = a.flatten()
d = sp.diags(b)
print(d)
print(d.shape)

[[1. 0. 5.]
 [0. 2. 0.]
 [0. 0. 3.]]
[6. 2. 3.]
[0.16666667 0.5        0.33333333]
  (0, 0)	0.16666666666666666
  (1, 1)	0.5
  (2, 2)	0.3333333333333333
(3, 3)


In [34]:
# 理解sp.coo_matrix的处理：
labels = encode_onehot(idx_features_labels[:, -1])
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}
edges_unordered = np.genfromtxt("{}{}.cites".format("./cora/", "cora"),dtype=np.int32)
# print(edges_unordered.shape)
# print(edges_unordered.flatten().shape)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
# 为了格式能成功转换,此处需要调用.flatten()改变shape,最后再转回来
print(edges_unordered)
print("-"*50)
print(edges)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)
# print(adj)

[[     35    1033]
 [     35  103482]
 [     35  103515]
 ...
 [ 853118 1140289]
 [ 853155  853118]
 [ 954315 1155073]]
--------------------------------------------------
[[ 163  402]
 [ 163  659]
 [ 163 1696]
 ...
 [1887 2258]
 [1902 1887]
 [ 837 1686]]


In [22]:
# from __future__ import division
# from __future__ import print_function
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

parser = argparse.ArgumentParser()
parser.add_argument("--no-cuda", action="store_true", default=False, help="Disables CUDA training.")
parser.add_argument("--fastmode", action="store_true", default=True, help="Validate during training pass.")
parser.add_argument("--seed", type=int, default=42, help="Random seed.")
parser.add_argument("--epochs", type=int, default=200, help="Number of epochs to train.")
parser.add_argument("--lr", type=float, default=0.01, help="Initial learning rate.")
parser.add_argument("--weight_decay", type=float, default=5e-4, help="Weight decay(L2 loss on parameters).")
parser.add_argument("--hidden", type=int, default=16, help="Number of hidden units")
parser.add_argument("--dropout", type=float, default=0.5, help="Dropout rate (1 - keep probability).")
# args = parser.parse_args()
args =parser.parse_known_args()[0]
# print(parser)
print(args)
args.cuda = not args.no_cuda and torch.cuda.is_available()
np.random.seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

adj, features, labels, idx_train, idx_val, idx_test = load_data()
model = GCN(nfeat=features.shape[1], nhid=args.hidden, nclass=labels.max().item()+1, dropout=args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        model.eval() # 注意：dropout会影响前向传播,从而影响预测结果
        output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print("Epoch:{:04d}".format(epoch+1), "loss_train:{:.4f}".format(loss_train.item()), "acc_train:{:.4f}".format(acc_train.item()), "loss_val:{:.4f}".format(loss_val.item()), "acc_val:{:.4f}".format(acc_val.item()), "time:{:.4f}".format(time.time()-t))

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:", "loss={:.4f}".format(loss_test.item()), "accuracy={:.4f}".format(acc_test.item()))

t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Total time:{:.4f}s".format(time.time()-t_total))

test()

Namespace(dropout=0.5, epochs=200, fastmode=True, hidden=16, lr=0.01, no_cuda=False, seed=42, weight_decay=0.0005)
Loading cora dataset...
Epoch:0001 loss_train:2.0336 acc_train:0.0714 loss_val:2.0350 acc_val:0.0933 time:0.0480
Epoch:0002 loss_train:2.0109 acc_train:0.0714 loss_val:2.0178 acc_val:0.1133 time:0.0440
Epoch:0003 loss_train:1.9999 acc_train:0.0857 loss_val:2.0020 acc_val:0.1000 time:0.0420
Epoch:0004 loss_train:1.9795 acc_train:0.0857 loss_val:1.9836 acc_val:0.1000 time:0.0400
Epoch:0005 loss_train:1.9621 acc_train:0.1000 loss_val:1.9706 acc_val:0.1100 time:0.0400
Epoch:0006 loss_train:1.9420 acc_train:0.1357 loss_val:1.9550 acc_val:0.1033 time:0.0370
Epoch:0007 loss_train:1.9257 acc_train:0.1571 loss_val:1.9398 acc_val:0.1567 time:0.0390
Epoch:0008 loss_train:1.9175 acc_train:0.1857 loss_val:1.9280 acc_val:0.2167 time:0.0406
Epoch:0009 loss_train:1.8994 acc_train:0.2929 loss_val:1.9171 acc_val:0.2333 time:0.0370
Epoch:0010 loss_train:1.8940 acc_train:0.2714 loss_val:1.906